# Flyte in a Notebook

Execute flyte workflows directly from a notebook.

In [1]:
%load_ext run_notebook_remote

The run_notebook_remote module is not an IPython extension.


In [2]:
import pandas as pd
from sklearn.datasets import load_wine
from sklearn.linear_model import LogisticRegression

from flytekit import task, workflow

Create tasks

In [3]:
@task
def get_data() -> pd.DataFrame:
    """Get the wine dataset."""
    return load_wine(as_frame=True).frame


@task
def process_data(data: pd.DataFrame) -> pd.DataFrame:
    """Simplify the task from a 3-class to a binary classification problem."""
    return data.assign(target=lambda x: x["target"].where(x["target"] == 0, 1))


@task
def train_model(data: pd.DataFrame, hyperparameters: dict) -> LogisticRegression:
    """Train a model on the wine dataset."""
    features = data.drop("target", axis="columns")
    target = data["target"]
    return LogisticRegression(max_iter=3000, **hyperparameters).fit(features, target)

Create a workflow

In [4]:
@workflow
def training_workflow(hyperparameters: dict) -> LogisticRegression:
    """Put all of the steps together into a single workflow."""
    data = get_data()
    processed_data = process_data(data=data)
    return train_model(
        data=processed_data,
        hyperparameters=hyperparameters,
    )

In [5]:
%notebook_run --remote training_workflow --hyperparameters '{"C": 0.1}'

Go to http://localhost:30080/console/projects/flytesnacks/domains/development/executions/fe29516afab8e417a801 to see execution in the console.
